In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec

# The first part, try to load jason file (not really useful, dont run it)

In [2]:
# list all files under folder 
import os
directory = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent'
for filename in os.listdir(directory):
    print(filename)

BuzzFeed_Fake_91-Webpage.json
BuzzFeed_Fake_69-Webpage.json
BuzzFeed_Fake_83-Webpage.json
BuzzFeed_Fake_29-Webpage.json
BuzzFeed_Fake_84-Webpage.json
BuzzFeed_Fake_18-Webpage.json
BuzzFeed_Fake_58-Webpage.json
BuzzFeed_Fake_85-Webpage.json
.DS_Store
BuzzFeed_Fake_28-Webpage.json
BuzzFeed_Fake_82-Webpage.json
BuzzFeed_Fake_68-Webpage.json
BuzzFeed_Fake_90-Webpage.json
BuzzFeed_Fake_59-Webpage.json
BuzzFeed_Fake_19-Webpage.json
BuzzFeed_Fake_13-Webpage.json
BuzzFeed_Fake_1-Webpage.json
BuzzFeed_Fake_46-Webpage.json
BuzzFeed_Fake_14-Webpage.json
BuzzFeed_Fake_41-Webpage.json
BuzzFeed_Fake_6-Webpage.json
BuzzFeed_Fake_54-Webpage.json
BuzzFeed_Fake_53-Webpage.json
BuzzFeed_Fake_30-Webpage.json
BuzzFeed_Fake_65-Webpage.json
BuzzFeed_Fake_37-Webpage.json
BuzzFeed_Fake_62-Webpage.json
BuzzFeed_Fake_88-Webpage.json
BuzzFeed_Fake_22-Webpage.json
BuzzFeed_Fake_77-Webpage.json
BuzzFeed_Fake_25-Webpage.json
BuzzFeed_Fake_70-Webpage.json
BuzzFeed_Fake_52-Webpage.json
BuzzFeed_Fake_55-Webpage.json
Bu

In [3]:
import json
#data=json.loads('BuzzFeed_Fake_1-Webpage.json')
path = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent/BuzzFeed_Fake_88-Webpage.json'
with open(path, 'r') as myfile:
    if len(myfile.readlines()) != 0:
        myfile.seek(0)
        data = json.load(myfile)

In [69]:
print(type(data))

<class 'dict'>


In [70]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()

In [71]:
df.head()

,top_img,text,authors,keywords,meta_data,canonical_link,images,title,url,summary,movies,publish_date,source
0,http://rightwingnews.com/wp-content/uploads/20...,Hillary’s TOP Donor Country Just Auctioned Off...,[Terresa Monroe-hamilton],[],"{'googlebot': 'noimageindex', 'og': {'site_nam...",http://rightwingnews.com/hillary-clinton-2/hil...,[http://1.gravatar.com/avatar/d35b77ff6c390071...,Hillary’s TOP Donor Country Just Auctioned Off...,http://rightwingnews.com/hillary-clinton-2/hil...,,[],{'$date': 1474912025000},http://rightwingnews.com


# read all files under same folder to dataframe (Start to Run)

below is the code to read all json file under a folder together Fake[BuzzFeed] 


In [2]:
import os
import json
myList = []
my_df  = pd.DataFrame()
directory = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent'
file_names = os.listdir(directory)
cleaned_file_name = [f for f in file_names if f.endswith('.json')]
#for i, filename in zip(range(1),cleaned_file_name):
for filename in cleaned_file_name:   
    with open(os.path.join(directory,filename),'r',encoding="utf8") as myfile:
        #print(filename)
        data = json.load(myfile) 
        myList.append(data)
    #with open(directory + '/' + filename,'r' ,encoding="utf8", errors='ignore') as myfile:
    #    if len(myfile.readlines()) != 0:
    #        myfile.seek(0)
    #        data = json.load(myfile)
    #        print(data)
    #        #data.append(json.load(myfile))
    #       #my_df = my_df.append(pd.DataFrame.from_dict(data, orient='index'))

In [3]:
# myList is a list of dictionary 
print(len(myList))

91


below is the code to read all json file under a folder together Real[BuzzFeed]

In [4]:
myList_real = []
my_df_real  = pd.DataFrame()
directory_real = '/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/RealNewsContent'
file_names_real = os.listdir(directory_real)
cleaned_file_name_real = [f for f in file_names_real if f.endswith('.json')]
#for i, filename in zip(range(1),cleaned_file_name):
for filename in cleaned_file_name_real:   
    with open(os.path.join(directory_real,filename),'r',encoding="utf8") as myfile:
        #print(filename)
        data = json.load(myfile) 
        myList_real.append(data)

In [5]:
print(len(myList_real))

91


The function to load glove model, the model includes 400,000 words and put these words in a dictionary Model, the dictionary like --- "a", [a vector of number]

In [6]:
# function load glove model
file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
      
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        if (word == "rahami"):
            print("rahami exist in the list in this case") # rahamim exist
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
          
model= loadGloveModel(file)

Loading Glove Model
Done. 400000  words loaded!


checkword is a function to check if a word in the news exist in the model, if model dosent contains this word, then return true --- in the next step, if dosen't contain, then make this word to a 0 vector 

In [7]:
# check if a word appears in the glove.6B.50d.txt or not , if not return true
def checkword (search_word):
    if search_word not in model:
        return (True)

In [8]:
print(checkword("rahami")) # rahami not exist in the glove dictionary(model), should return true

True


In [9]:
# check if a word appears in the glove.6B.50d.txt, if the word not appears in the txt file, then set the vector to 0
'''
def checkword (search_word):
    file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
    news_file = open(file)
    each_line = news_file.readlines()
    for el in each_line:
        line_split = el.split()
        word = line_split[0]
        if(word != search_word):
            #print("it doesn't exist", search_word)
            #per_word_vector = np.empty(50)
            #per_word_vector.fill(0)
            return (True)  
'''

'\ndef checkword (search_word):\n    file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"\n    news_file = open(file)\n    each_line = news_file.readlines()\n    for el in each_line:\n        line_split = el.split()\n        word = line_split[0]\n        if(word != search_word):\n            #print("it doesn\'t exist", search_word)\n            #per_word_vector = np.empty(50)\n            #per_word_vector.fill(0)\n            return (True)  \n'

In [10]:
#print(checkword("rahami"))
#print(model['hello'])
#print(model['rahami'])


In [11]:
import re 
per_new_text= []
news_text = []
temp = []
file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
#print(len(myList))
for i in range (0,len(myList)): # i from 0 to 90
    #print(i)
    new = myList[i]
    news_content = new['text']
    new_content_token = re.sub(r'[^\w]', ' ', news_content).lower().split()
    new_content_token = [ word for word in new_content_token if word not in stopwords.words('english')]
#print(new_content_token)
#    print(new_content_token[0])
#    print(model[new_content_token[0]])
#    print(model[new_content_token[1]])
#    print(model[new_content_token[2]])
    for word in new_content_token:
        #print(word)
        if (checkword(word) == True): # if a word not appears in the glove txt, then set the vector to 0 
            per_word_vector = np.empty(50)
            per_word_vector.fill(0) 
        else:
            per_word_vector = model[word]
        per_new_text.append(per_word_vector)
    news_text.append(per_new_text)
            
        #print(per_word_vector)
        #print(per_word_vector.shape)   
        #print(per_word_vector)
        #print(news_text)
#print(len(new))

In [13]:
#print(len(news_text)) -- 91
print((len(news_text[90]))) #- 23021 
print(news_text[90][0])  # check 
print((news_text[90][1]))


23021
[ 0.95281   -0.20608    0.55618   -0.46323    0.73354    0.029137
 -0.19367   -0.090066  -0.22958   -0.19058   -0.34857   -1.0231
  0.743     -0.5489     0.88484   -0.14051    0.0040139  0.58448
  0.10767   -0.44657   -0.43205    0.9868     0.78288    0.51513
  0.85788   -1.7713    -0.88259   -0.59728    0.084934  -0.48112
  3.9678     0.8893    -0.27064   -0.44094   -0.26213    0.085597
  0.022099  -0.58376    0.10908    0.77973   -0.95447    0.40482
  0.8941     0.65251    0.39858    0.20884   -1.3281    -0.10882
 -0.22822   -0.46303  ]
[ 0.23918  -0.096261  0.024736 -0.592     0.51256  -0.035554 -1.1253
  0.48754  -0.12025  -0.37303   0.30812  -0.081929 -0.14237   0.11731
  0.65941   0.28794  -0.66977   0.049251 -0.31647  -1.0925   -0.25429
  0.39473   0.16178   0.10083   0.85331  -1.1566   -0.76104   0.95558
  0.55682  -0.36111   1.4445    0.15637   0.66717  -1.0544   -0.019116
  0.1055    0.12556   0.23093   0.41279   0.24118  -0.30911   0.38222
 -0.30777   0.5155    0.54437

convert BuzzFed real new to vector 

In [14]:
import re 
per_new_text_real= []
news_text_real = []
#print(len(myList))
for i in range (0,len(myList_real)): # i from 0 to 90
    #print(i)
    new_real = myList_real[i]
    news_content_real = new_real['text']
    new_content_token_real = re.sub(r'[^\w]', ' ', news_content_real).lower().split()
    new_content_token_real = [ word for word in new_content_token_real if word not in stopwords.words('english')]
    for word in new_content_token_real:
        if (checkword(word) == True): # if a word not appears in the glove txt, then set the vector to 0 
            per_word_vector_real = np.empty(50)
            per_word_vector_real.fill(0) 
        else:
            per_word_vector_real = model[word]
            per_new_text_real.append(per_word_vector_real)
    news_text_real.append(per_new_text_real)

In [15]:
print((len(news_text_real[90]))) #- 30481
print(news_text_real[90][0])  # check 
print((news_text_real[90][1]))

30481
[ 1.4675e-01  1.1692e+00  6.9416e-01 -6.1429e-02 -1.3677e-01  4.2015e-01
 -7.1600e-01  1.9014e-02 -5.2896e-01 -8.3643e-01 -1.8561e+00 -1.8324e-01
  5.7648e-02 -3.1188e-01  2.4997e-02  4.5878e-02 -9.8728e-02 -2.1451e-01
  1.4298e-01 -8.0809e-03 -1.4569e-01  3.8326e-01  6.3811e-01 -4.6426e-01
  1.0953e+00 -2.1500e+00 -1.8462e-01  1.7380e-01 -5.0607e-01  5.7719e-04
  5.2828e-01  6.6850e-01 -8.9692e-01 -3.4346e-01 -1.5456e-01 -9.7313e-01
 -6.9441e-01  5.9201e-01 -1.2194e+00 -1.3469e+00 -2.5691e-01  3.4537e-01
 -4.3824e-01 -9.6233e-02  2.9882e-01 -2.9174e-01 -4.7201e-01 -3.2221e-01
  7.9279e-02  5.9419e-01]
[ 0.23158   0.69964   0.43878  -0.31633   0.18509   0.45519  -0.52914
  0.13019  -0.6016   -0.53644  -1.8771   -0.32314  -0.13149   0.05427
  0.24486  -0.037868 -0.34769  -0.60215   0.72329  -0.47918   0.15473
  0.4589    0.37807  -1.0418    0.43743  -2.6996    0.2478    0.68145
 -0.23022  -0.024437  2.133     0.46905  -1.1284   -1.0115   -0.3608
 -0.89718  -0.47746   0.46726  -0.8

# add fake news vecotr and real new vector to one list 

In [69]:
BuzzFeed_news = []
BuzzFeed_news.extend(news_text)  # fake news 
BuzzFeed_news.extend(news_text_real) # real news
BuzzFeed_news = np.array(BuzzFeed_news)

In [70]:
#[buzzfed[each_news[each_words]]]
print(len(BuzzFeed_news))
print(type(BuzzFeed_news)) # buzz feed is a list of list, the first 91 is fake new snd last last 91 is real news
print(type(BuzzFeed_news[0])) # a list 
print(len(BuzzFeed_news[0])) # numpy array # 23021 words in the last news 

182
<class 'numpy.ndarray'>
<class 'list'>
23021


# create response variable y fake is 1 and real is 0, y is a list

In [74]:
# create a list to hold the response variable fake and real (1 and 0), the first 91 is fake and last 91 is real
fake_y = [1] * 91
real_y = [0]*91
fake_real = []
fake_real.extend(fake_y)
fake_real.extend(real_y)
print(len(fake_real))

# one hot encode the output variable
BuzzFeed_Y = np.array (fake_real)
print(type(BuzzFeed_Y))

182
<class 'numpy.ndarray'>


# Model

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(BuzzFeed_news)))
model.add(Dense(BuzzFeed_Y, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=batch_size, verbose=2)


# Some examples or some useless code 

In [49]:
# example of the content of the first news
# from the list of dictionary , delete all \n or \nn for one new
news_text = []
new = myList[0]
news_content = new['text']
result = []
#print(news_content) # sentence
new_content_token= news_content.replace('\n', ' ').lower().split()
#print(new_content_token) # a list of words
# remove stopwords word like I , am , my, the ...
new_content_token = [ word for word in new_content_token if word not in stopwords.words('english')]
#print(new_content_token)
# remove wired charactor like ?, ..., "', multiple X, 101, #, :, it's, we'll
for word in new_content_token: 
    sc = "….?’,“”'#!×:"
    for char in sc:
        word = word.replace(char," ")  
    word = word.split()
    #print(word)
    result.extend(word)
print(result)
for element in result:
    #print(element)
    per_word_vector = model[element]
    
    #print(model[element])
    #print(type(model[element])) # type of per_word_vector is numpy.ndarray 
    
    
# convert each word in new_content_token to vector using word2vec or glove vector
#print(model['hello'])
#print(len(model['hello']))
#for word in new_content_token:
    #print(model[word])
    

['people', 'noticed', 'something', 'odd', 'hillary', 's', 'outfit', 'debate', 'last', 'night', 'there', 's', 'lot', 'could', 'discussed', 'last', 'night', 's', 'debate', 'like', 'unfair', 'debate', 'moderator', 'asking', 'questions', 'help', 'hillary', 'that', 's', 'main', 'concern', 'right', 'now', 'one', 'thing', 'baffled', 'many', 'hillary', 'didn', 't', 'appear', 'usually', 'stupid', 'coughing', 'self', 'ready', 'answers', 'detailed', 'facts', 'almost', 'bizarre', 'sputter', 'reach', 'glass', 'water', 'odd', 'everyone', 'grandma', 'talking', 'latest', 'question', 'swirling', 'around', 'social', 'media', 'pictures', 'appeared', 'show', 'hillary', 'kind', 'flesh-colored', 'device', 'embedded', 'inside', 'ear', 'is', 'earpiece', 'wearing', 'something', 'fashionable', 'old', 'ladies', 'twitter', 'may', 'figured', 'answer', 'do', 'see', 'it', 'certainly', 'wouldn', 't', 'first', 'time', 'hillary', 'use', 'device', 'help', 'think', 'could', 'trick', 'light', 'shadows', 'puffy', 'zipper',

In [7]:
print(type(model[element]))

<class 'numpy.ndarray'>


In [16]:
# example to delete ' and s
a = 'hillary’s'
b = "’s"
for char in b:
    a = a.replace(char,"")
    print(a)

hillarys
hillary


In [13]:
#print(new_content_token)
print(type(new_content_token))

<class 'list'>
people


In [58]:
data.keys()

dict_keys(['top_img', 'text', 'authors', 'keywords', 'meta_data', 'canonical_link', 'images', 'title', 'url', 'summary', 'movies', 'publish_date', 'source'])

In [56]:
# concatenate string
new = directory + '/'+ 'BuzzFeed_Fake_80-Webpage.json'
print(new)

/Users/moranwang/Desktop/SummerQuarterDS/Capstone/FakeNewsNet/Data/BuzzFeed/FakeNewsContent/BuzzFeed_Fake_80-Webpage.json


# Example1 : convert text to vector Example

In [2]:
#word2vec example1/ using own word embeddings
from gensim.models import Word2Vec

# define training data
sentences = [['this', 'is', 'the', 'first', 'sentence', 'for!', 'word2vec'],
             ['this', 'is', 'the', 'second', 'sentence'],
             ['yet', 'another', 'sentence'],
             ['one', 'more', 'sentence'],
             ['and', 'the', 'final', 'sentence']]
# train model
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print('model is ',model)
# summarize vocabulary,all words that appears in the sentences
words = list(model.wv.vocab)
print('words is ',words)
# access vector for one word
print('access vecotr for one word', model['sentence'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

from sklearn.decomposition import PCA
from matplotlib import pyplot
# plot 
X = model[model.wv.vocab]

pca = PCA(n_components=2)
result = pca.fit_transform(X)

pyplot.scatter(result[:, 0], result[:, 1])

words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

model is  Word2Vec(vocab=14, size=100, alpha=0.025)
words is  ['this', 'is', 'the', 'first', 'sentence', 'for!', 'word2vec', 'second', 'yet', 'another', 'one', 'more', 'and', 'final']
access vecotr for one word [ 4.14259126e-03  2.79425620e-03 -2.77865003e-03 -2.06513098e-03
 -3.35403159e-03  2.78954208e-03  3.93661810e-03  1.55871559e-03
 -1.44100119e-03 -4.32085153e-03  4.30899672e-03 -4.62976005e-03
 -3.79096786e-03  2.00172933e-03 -4.60096542e-03 -1.31963647e-03
  2.37290107e-04  1.49146968e-03  2.41897837e-03  2.93111708e-03
  3.15664662e-03 -2.23306101e-03 -5.75355079e-04  6.63872808e-04
  4.13089339e-03  6.29035581e-04  1.40708033e-03 -6.88619097e-04
  4.30145860e-03 -1.68555172e-03  2.99498509e-03  3.51108215e-03
 -1.11267739e-03  2.78331194e-04  3.57962656e-03  1.52858009e-03
  1.35704526e-03  4.27913677e-04 -3.05608939e-03 -3.82844337e-05
  3.61788506e-03 -3.90263507e-03  3.21124983e-03  3.75231705e-03
 -1.91865454e-03  3.26324417e-03 -2.85139750e-03  4.45809821e-03
 -3.87078

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# Example2: golve / not run this code again

In [5]:
file = "/Users/moranwang/Desktop/SummerQuarterDS/glove/glove.6B.50d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    
     
    with open(gloveFile, encoding="utf8" ) as f:
       content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model
     
     
model= loadGloveModel(file)   
 
print (model['hello'])

Loading Glove Model
Done. 400000  words loaded!
[-0.38497   0.80092   0.064106 -0.28355  -0.026759 -0.34532  -0.64253
 -0.11729  -0.33257   0.55243  -0.087813  0.9035    0.47102   0.56657
  0.6985   -0.35229  -0.86542   0.90573   0.03576  -0.071705 -0.12327
  0.54923   0.47005   0.35572   1.2611   -0.67581  -0.94983   0.68666
  0.3871   -1.3492    0.63512   0.46416  -0.48814   0.83827  -0.9246
 -0.33722   0.53741  -1.0616   -0.081403 -0.67111   0.30923  -0.3923
 -0.55002  -0.68827   0.58049  -0.11626   0.013139 -0.57654   0.048833
  0.67204 ]


# Example of using  LSTM Recurrent Neural Networks in Python with Keras

Learn the Alphabet

In [93]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [94]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [97]:
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
print(char_to_int)
print(int_to_char)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [101]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    print(seq_in, '->', seq_out)
print(dataX)
print(dataY)

A -> B
B -> C
C -> D
D -> E
E -> F
F -> G
G -> H
H -> I
I -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> Q
Q -> R
R -> S
S -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z
[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15], [16], [17], [18], [19], [20], [21], [22], [23], [24]]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [103]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

[[[ 0]]

 [[ 1]]

 [[ 2]]

 [[ 3]]

 [[ 4]]

 [[ 5]]

 [[ 6]]

 [[ 7]]

 [[ 8]]

 [[ 9]]

 [[10]]

 [[11]]

 [[12]]

 [[13]]

 [[14]]

 [[15]]

 [[16]]

 [[17]]

 [[18]]

 [[19]]

 [[20]]

 [[21]]

 [[22]]

 [[23]]

 [[24]]]


In [107]:
# normalize
X = X / float(len(alphabet))
print(X)

[[[0.        ]]

 [[0.00147929]]

 [[0.00295858]]

 [[0.00443787]]

 [[0.00591716]]

 [[0.00739645]]

 [[0.00887574]]

 [[0.01035503]]

 [[0.01183432]]

 [[0.01331361]]

 [[0.0147929 ]]

 [[0.01627219]]

 [[0.01775148]]

 [[0.01923077]]

 [[0.02071006]]

 [[0.02218935]]

 [[0.02366864]]

 [[0.02514793]]

 [[0.02662722]]

 [[0.02810651]]

 [[0.0295858 ]]

 [[0.03106509]]

 [[0.03254438]]

 [[0.03402367]]

 [[0.03550296]]]


In [106]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(y)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0.

LSTM Models:
1. Naive LSTM for Learning One-Char to One-Char Mapping (have problem)
2. Naive LSTM for a Three-Char Feature Window to One-Char Mapping (have problem)
3. Naive LSTM for a Three-Char Time Step Window to One-Char Mapping (have problem)
4. LSTM State Within A Batch (good model) 
5. Stateful LSTM for a One-Char to One-Char Mapping
6. LSTM with Variable-Length Input to One-Char Output


In [21]:
# We will frame the problem as a random collection of one-letter input to one-letter output pairs. 
# As we will see this is a difficult framing of the problem for the LSTM to learn.

# LSTM with Variable-Length Input to One-Char Output

learns random subsequences of the alphabet and an effort to build a model that can be given arbitrary letters or subsequences of letters and predict the next letter in the alphabet.



In [48]:
# LSTM with Variable Length Input Sequences to One Character Output
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from theano.tensor.shared_randomstreams import RandomStreams

In [67]:
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
num_inputs = 50
max_len = 5
dataX = []
dataY = []
for i in range(num_inputs):
    start = numpy.random.randint(len(alphabet)-2)
    end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1))
    sequence_in = alphabet[start:end+1]
    sequence_out = alphabet[end + 1]
    dataX.append([char_to_int[char] for char in sequence_in])
    dataY.append(char_to_int[sequence_out])
    print (sequence_in, '->', sequence_out)
# convert list of lists to array and pad sequences if needed
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(X, (X.shape[0], max_len, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
batch_size = 1
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], 1)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=batch_size, verbose=2)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
for i in range(20):
    pattern_index = numpy.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = pad_sequences([pattern], maxlen=max_len, dtype='float32')
    x = numpy.reshape(x, (1, max_len, 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print (seq_in, "->", result)

PQRST -> U
W -> X
O -> P
OPQ -> R
IJKLM -> N
QRSTU -> V
ABCD -> E
X -> Y
GHIJ -> K
M -> N
XY -> Z
QRST -> U
ABC -> D
JKLMN -> O
OP -> Q
XY -> Z
D -> E
T -> U
B -> C
QRSTU -> V
HIJ -> K
JKLM -> N
ABCDE -> F
X -> Y
V -> W
DE -> F
DEFG -> H
BCDE -> F
EFGH -> I
BCDE -> F
FG -> H
RST -> U
TUV -> W
STUV -> W
LMN -> O
P -> Q
MNOP -> Q
JK -> L
MNOP -> Q
OPQRS -> T
UVWXY -> Z
PQRS -> T
D -> E
EFGH -> I
IJK -> L
WX -> Y
STUV -> W
MNOPQ -> R
P -> Q
WXY -> Z
Epoch 1/500
 - 1s - loss: 3.2632 - acc: 0.0000e+00
Epoch 2/500
 - 0s - loss: 3.2132 - acc: 0.0800
Epoch 3/500
 - 0s - loss: 3.1570 - acc: 0.0800
Epoch 4/500
 - 0s - loss: 3.0627 - acc: 0.0800
Epoch 5/500
 - 0s - loss: 2.9682 - acc: 0.0800
Epoch 6/500
 - 0s - loss: 2.9126 - acc: 0.0800
Epoch 7/500
 - 0s - loss: 2.8873 - acc: 0.0800
Epoch 8/500
 - 0s - loss: 2.8627 - acc: 0.0800
Epoch 9/500
 - 0s - loss: 2.8430 - acc: 0.0800
Epoch 10/500
 - 0s - loss: 2.8245 - acc: 0.1000
Epoch 11/500
 - 0s - loss: 2.7976 - acc: 0.0800
Epoch 12/500
 - 0s - loss:

 - 0s - loss: 1.0061 - acc: 0.7000
Epoch 162/500
 - 0s - loss: 1.0111 - acc: 0.6200
Epoch 163/500
 - 0s - loss: 1.0090 - acc: 0.6200
Epoch 164/500
 - 0s - loss: 0.9907 - acc: 0.6800
Epoch 165/500
 - 0s - loss: 0.9907 - acc: 0.6800
Epoch 166/500
 - 0s - loss: 0.9993 - acc: 0.6600
Epoch 167/500
 - 0s - loss: 0.9663 - acc: 0.6600
Epoch 168/500
 - 0s - loss: 0.9744 - acc: 0.6600
Epoch 169/500
 - 0s - loss: 0.9739 - acc: 0.6800
Epoch 170/500
 - 0s - loss: 0.9613 - acc: 0.7200
Epoch 171/500
 - 0s - loss: 0.9723 - acc: 0.6800
Epoch 172/500
 - 0s - loss: 0.9678 - acc: 0.6800
Epoch 173/500
 - 0s - loss: 0.9588 - acc: 0.6800
Epoch 174/500
 - 0s - loss: 0.9615 - acc: 0.6200
Epoch 175/500
 - 0s - loss: 0.9477 - acc: 0.6800
Epoch 176/500
 - 0s - loss: 0.9478 - acc: 0.6800
Epoch 177/500
 - 0s - loss: 0.9701 - acc: 0.6800
Epoch 178/500
 - 0s - loss: 0.9578 - acc: 0.6600
Epoch 179/500
 - 0s - loss: 0.9358 - acc: 0.7000
Epoch 180/500
 - 0s - loss: 0.9453 - acc: 0.6600
Epoch 181/500
 - 0s - loss: 0.9217

KeyboardInterrupt: 